<a href="https://colab.research.google.com/github/FelipeGomide/TP2_IBD/blob/main/TP2_IBD_Testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando bibliotecas

In [ ]:
import pandas as pd
import sqlite3
import io

#Monto meu google drive no colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Quero mudar isso, colocar tudo no github

#Abertura dos arquivos

In [ ]:
#Abro o csv e gero um dataframe com suas informações
df = pd.read_csv('/content/drive/MyDrive/IBD/source/20230424-tabelapanespecies.csv', encoding='unicode_escape', sep=';')

#Renomeio alguma colunas
df = df.rename(
    columns={
        "idTaxon": "IDEspecie",
        "taxonNome": "Nome_Cientifico",
    }
)

#Removo a coluna 315, que tinha uma espécie apagada
df = df.drop(315)

#Removo a coluna pan_nome, que é redundante
df = df.drop(columns=['pan_nome'])

#Corrijo os nomes de 534 e 856
df = df.replace({'Crypturellus noctivagus noctivagus Crypturellus noctivagus':'Crypturellus noctivagus noctivagus'})

#Quebro a coluna nome científico em 3 nomes
df[['Nome_genero', 'Epiteto_especifico', 'subespecie']] = df['Nome_Cientifico'].str.split(expand=True)

#Mudo a ordem das colunas
df = df.iloc[:, [2,0,1,3,4,5]]

#Possíveis mudanças, excluir IDEspecie, que parece inútil, excluir nome_cientifico

df

#Com problemas
# 315, não é mais subespécie
# 534 e 856, Crypturellus noctivagus noctivagus Crypturellus noctivagus

,idPan,IDEspecie,Nome_Cientifico,Nome_genero,Epiteto_especifico,subespecie
0,CEMAVE_27.02,10488,Cyanopsitta spixii,Cyanopsitta,spixii,None
1,RAN_10.02,7915,Adelophryne maranguapensis,Adelophryne,maranguapensis,None
2,RAN_10.02,7935,Hylomantis granulosa,Hylomantis,granulosa,None
3,RAN_10.02,12115,Glaucomastix abaetensis,Glaucomastix,abaetensis,None
4,RAN_10.02,12121,Ameivula nativo,Ameivula,nativo,None
...,...,...,...,...,...,...
2391,CEPTA_09.01,6333,Bunocephalus larai,Bunocephalus,larai,None
2392,CEPTA_09.01,4391,Characidium oiticicai,Characidium,oiticicai,None
2393,CEPTA_09.01,7368,Pseudopimelodus mangurus,Pseudopimelodus,mangurus,None
2394,CEPTA_09.01,7191,Rhinelepis aspera,Rhinelepis,aspera,None


#Colocando dados no sql e criando .dump

In [ ]:
#Crio a database e a conexão
conn = sqlite3.connect('test_database')
c = conn.cursor()

#Crio o esquema para a tabela
c.execute('CREATE TABLE IF NOT EXISTS especies (id_expecie number, nome_cientifico text, idPan text, pan_nome text)')
conn.commit()

#Passo as informações do dataframe pandas para a tabela sql
df.to_sql('especies', conn, if_exists='replace', index = False)

#Crio o dump em sql
with open('dump_mexido.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)
conn.close()

#Contando valor de instâncias abrangência

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IBD/source/20230424-tabelapandados.csv', encoding='ISO-8859-1', sep=';')

# df = df.iloc[:, [4,5]]
# df = df.drop_duplicates()
# df = df.reset_index()
df.head()

,idPan,panNome,panNomeCompleto,panNomeFantasia,panAbrangenciaTaxonomica,panAbrangenciaGeografica,panCiclo,panStatus,panCentro,panInicioData,panInicioAno,Data da portaria vigente do PAN,Data do início da vigência do PAN na portaria \n(último artigo da portaria),panFimData,panFimAno,panPortaria,panProcesso,panLogo,panSite
0,CBC_01.01,(Finalizado) Lepidópteros,Plano de Ação Nacional para a Conservação de L...,Lepidópteros,Intraclasse,Nacional,1º ciclo,Finalizado,CBC,27/08/2010,2010,27/08/2010,27/08/2010,01/12/2015,2015.0,Portaria 92 (27/08/2010),02070.002646/2010-11,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
1,CBC_02.01,Insetos Polinizadores,Plano de Ação Nacional para a Conservação Plan...,Insetos Polinizadores,Intraclasse,Nacional,1º ciclo,Em execução,CBC,05/12/2022,2022,05/12/2022,02/01/2022,31/12/2027,2027.0,Portaira 1145 (05/12/2022),02176.000061/2019-17,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
2,CECAV_01.01,(Finalizado) Morceguinho-do-cerrado,Plano de Ação Nacional para Conservação do Mor...,Morceguinho-do-cerrado,Monoespecífico,Bioma,1º ciclo,Finalizado,CECAV,27/08/2010,2010,27/08/2010,27/08/2010,01/09/2015,2015.0,Portaria 89 (27/08/2010),02070.003685/2009-94,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
3,CECAV_02.01,(Finalizado) Cavernas do Rio São Francisco,Plano de Ação Nacional para a Conservação do P...,Cavernas do Rio São Francisco,Multiclasse,Ecossistema,1º ciclo,Finalizado,CECAV,17/02/2012,2012,17/02/2012,17/02/2017,01/02/2017,2017.0,Portaria 18 (17/02/2012),02070.002654/2010-50,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
4,CECAV_03.01,Cavernas do Brasil,Plano de Ação Nacional para a Conservação do P...,Cavernas do Brasil,Multiclasse,Ecossistema,1º ciclo,Em execução,CECAV,09/08/2022,2022,09/08/2022,01/09/2022,01/08/2027,2027.0,Portaria 646 (09/08/2022),02667.000076/2019-45,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
